In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [4]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
243,I love Umberto Lenzi's cop movies -- ROME ARME...,negative
345,Routine suspense yarn about a sociopath (Dillo...,negative
897,This is the Columbo that got directed by Steve...,positive
225,This second film is just as interesting as the...,positive
512,I enjoyed every moment of this beautiful film ...,positive


In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7964\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [6]:
df = normalize_text(df)
df.head()

,review,sentiment
243,love umberto lenzi s cop movie rome armed teet...,negative
345,routine suspense yarn sociopath dillon give sp...,negative
897,columbo got directed steven spielberg early po...,positive
225,second film interesting previous one except su...,positive
512,enjoyed every moment beautiful film intrigued ...,positive


In [7]:
df['sentiment'].value_counts()

sentiment
negative    261
positive    239
Name: count, dtype: int64

In [8]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
243,love umberto lenzi s cop movie rome armed teet...,0
345,routine suspense yarn sociopath dillon give sp...,0
897,columbo got directed steven spielberg early po...,1
225,second film interesting previous one except su...,1
512,enjoyed every moment beautiful film intrigued ...,1


In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [11]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/rushikeshnayakavadi/Sentiment-Analysis.mlflow')
dagshub.init(repo_owner='rushikeshnayakavadi', repo_name='Sentiment-Analysis', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as rushikeshnayakavadi

Initialized MLflow to track repo "rushikeshnayakavadi/Sentiment-Analysis"

Repository rushikeshnayakavadi/Sentiment-Analysis initialized!

2025/03/25 23:43:31 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/263fbc796f7b4662976a45279d530c35', creation_time=1742926411566, experiment_id='0', last_update_time=1742926411566, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [14]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-03-25 23:43:40,477 - INFO - Starting MLflow run...
2025-03-25 23:43:41,484 - INFO - Logging preprocessing parameters...
2025-03-25 23:43:43,027 - INFO - Initializing Logistic Regression model...
2025-03-25 23:43:43,030 - INFO - Fitting the model...
2025-03-25 23:43:43,127 - INFO - Model training complete.
2025-03-25 23:43:43,129 - INFO - Logging model parameters...
2025-03-25 23:43:43,613 - INFO - Making predictions...
2025-03-25 23:43:43,617 - INFO - Calculating evaluation metrics...
2025-03-25 23:43:43,648 - INFO - Logging evaluation metrics...
2025-03-25 23:43:45,656 - INFO - Saving and logging the model...
2025/03/25 23:44:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-25 23:44:17,220 - INFO - Model training and logging completed in 35.74 seconds.
2025-03-25 23:44:17,224 - INFO - Accuracy: 0.624
2025-03-25 23:44:17,224 - INFO - Precision: 

🏃 View run treasured-trout-262 at: https://dagshub.com/rushikeshnayakavadi/Sentiment-Analysis.mlflow/#/experiments/0/runs/707fb06b0a614e23b5565cd494a35704
🧪 View experiment at: https://dagshub.com/rushikeshnayakavadi/Sentiment-Analysis.mlflow/#/experiments/0
